In [1]:
from andi_datasets.models_phenom import models_phenom
from andi_datasets.analysis import *

from helpersMSD import *
from helpersGeneration import *
from helpersPlot import *

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

c:\ProgramData\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [ ]:
# number of time steps per trajectory (frames)
nFrames = 20
# number of trajectories
# Length of box (pixels), set to 0 to make them start at 0,0
L = 0 
# diffusion coefficient (pixels^2 / dt)
# conversion to real numbers (given by andi): 0.1px^2/dt = 0.01 micro_m ^2 /s for dt = 0.100ms 
D = 1

alpha = 1
output_size = 30



nposframe = 10


# 0.1 px^2 / 100ms = 1 px^2/1s = 0.01 micro_m^2/s 

trajs, label = models_phenom().single_state(1,nFrames * nposframe,Ds=D/(nposframe**2),alphas=alpha)
trajs = trajs.transpose(1,0,2)
videos = trajectories_to_video(trajs,nposframe,image_props={"upsampling_factor":10,
                                            #"background_intensity": [0,0],
                                            "particle_intensity": [500,20],
                                            #"resolution": 200e-9,
                                            "output_size":output_size,
                                              })
print(videos.shape)
play_video(videos[0])

nposframe2 = 25

trajs2, label2 = models_phenom().single_state(1,nFrames * nposframe2,Ds=D/(nposframe2**2),alphas=alpha)
trajs2 = trajs2.transpose(1,0,2)
videos2 = trajectories_to_video(trajs2,nposframe2,image_props={"upsampling_factor":10,
                                            #"background_intensity": [0,0],
                                            "particle_intensity": [500,20],
                                            #"resolution": 200e-9,
                                            "output_size":output_size,
                                              })
print(videos2.shape)
play_video(videos2[0])


(1, 20, 11, 11)
vmin: 52.0 vmax: 550.0 mean: 114.31


In [16]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

def animate_particle_trajectory(trajectory, nframes, points_per_frame=None, fps=5, save_path=None):
    """
    Creates an animation of a particle's trajectory, showing one subtrajectory per frame.
    No axis labels or ticks for a clean visualization.
    
    Parameters:
    - trajectory: np.ndarray of shape (N, 2), where N is the total number of points.
                 Each row represents the (x, y) coordinates of the particle.
    - nframes: int, number of frames to divide the trajectory into.
    - points_per_frame: int, optional, number of points to show in each frame.
                       If None, it's calculated as len(trajectory) // nframes.
    - fps: int, frames per second for the animation.
    
    Returns:
    - HTML animation that can be displayed in a Jupyter notebook.
    """
    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(5, 5))
    
    # Calculate points per frame if not provided
    if points_per_frame is None:
        points_per_frame = len(trajectory) // nframes
    
    # Set consistent axis limits
    x_min, x_max = np.min(trajectory[:, 0]), np.max(trajectory[:, 0])
    y_min, y_max = np.min(-trajectory[:, 1]), np.max(-trajectory[:, 1])
    
    # Find the maximum absolute value to create a symmetric plot
    max_extent = max(
        abs(x_min), 
        abs(x_max), 
        abs(y_min), 
        abs(y_max)
    )
    
    # Add some padding to the limits
    max_extent = max_extent * 1.1  # Add 10% padding
    
    # Set symmetric limits centered at origin
    ax.set_xlim([-max_extent, max_extent])
    ax.set_ylim([-max_extent, max_extent])
    
    # Turn off all axis labeling
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    

    # Set up plot elements
    ax.set_aspect('equal')
    
    # Initialize empty line and text objects
    line, = ax.plot([], [], lw=2)
    frame_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=12)
    
    # Initialize empty arrays for accumulated trajectory
    x_data, y_data = [], []
    
    def init():
        """Initialize animation"""
        line.set_data([], [])
        frame_text.set_text('')
        return line, frame_text
    
    def update(frame):
        """Update animation for each frame"""
        # Calculate start and end indices for this frame
        start = frame * points_per_frame
        end = min(start + points_per_frame, len(trajectory))
        
        # Get the current trajectory segment
        current_segment = trajectory[start:end]
        
        # Append to accumulated data
        if frame == 0:
            x_data.clear()
            y_data.clear()
        
        x_data.extend(current_segment[:, 0])
        y_data.extend(-current_segment[:, 1])  # Negative y to match your original plot
        
        # Update the line data
        line.set_data(x_data, y_data)
        
        # Update the frame text
        frame_text.set_text(f'Frame {frame + 1}/{nframes}')
        
        return line, frame_text
    
    # Create the animation
    ani = animation.FuncAnimation(fig, update, frames=nframes,
                                 init_func=init, blit=True, interval=1000/fps)
    # Save the animation if a save path is provided
    if save_path:
        if save_path.endswith('.mp4'):
            # Use FFMpegWriter for MP4 files (requires FFmpeg installed)
            writer = animation.FFMpegWriter(fps=fps, metadata=dict(artist='Me'), bitrate=1800)
            ani.save(save_path, writer=writer)
            print(f"Animation saved to {save_path}")
        elif save_path.endswith('.gif'):
            # Use PillowWriter for GIF files
            writer = animation.PillowWriter(fps=fps)
            ani.save(save_path, writer=writer)
            print(f"Animation saved to {save_path}")
        else:
            print("Unsupported file format. Use .mp4 or .gif extension.")
    plt.close()  # Prevent displaying the static plot
    display(HTML(ani.to_jshtml()))

# Example usage:


In [29]:
# number of time steps per trajectory (frames)
nFrames = 20
# number of trajectories
# Length of box (pixels), set to 0 to make them start at 0,0
L = 0 
# diffusion coefficient (pixels^2 / dt)
# conversion to real numbers (given by andi): 0.1px^2/dt = 0.01 micro_m ^2 /s for dt = 0.100ms 
D = 1

alpha = 1.95
output_size = 50



nposframe = 10


# 0.1 px^2 / 100ms = 1 px^2/1s = 0.01 micro_m^2/s 

trajs, label = models_phenom().single_state(1,nFrames * nposframe,Ds=D/(nposframe**2),alphas=alpha)
trajs = trajs.transpose(1,0,2)
videos = trajectories_to_video(trajs,nposframe,center=False,image_props={"upsampling_factor":10,
                                            #"background_intensity": [0,0],
                                            "particle_intensity": [300,20],
                                            #"resolution": 200e-9,
                                            "output_size":output_size,
                                              })
animate_particle_trajectory(trajs[0], nFrames, save_path=None)
play_video(videos[0])

videos2 = trajectories_to_video(trajs,nposframe,center=True,image_props={"upsampling_factor":10,
                                            #"background_intensity": [0,0],
                                            "particle_intensity": [300,20],
                                            #"resolution": 200e-9,
                                            "output_size":7,
                                              })
play_video(videos2[0])


vmin: 48.0 vmax: 381.0 mean: 100.46


vmin: 63.0 vmax: 374.0 mean: 121.88
